







以下のコードが実験に使っていたコードになります。ただ、データやグラフについては、まだ学会などで発表していないため、公開できない状態とはなっています。


The following code is the code that was used for the experiment. However, the data and graphs have not yet been presented at conferences, etc., so they cannot be released to the public.

In [ ]:
# NumPy
import numpy as np

# Dataframe operations
import pandas as pd

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# import japanize_matplotlib

import datetime

# import operator

In [ ]:
# このセルを実行したあとどうすればいいかがわからない方は、別途配布している。GoogleColabにおけるドライブのマウントの仕方と、Google Drive上でのフォルダ構造の説明のpdfファイルをご覧ください。

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ニジマス(rainbow trout)

In [ ]:
class Model_rainbowtrout:

  def __init__(self, name, date):
    self.periods = []
    self.dates = []
    self.weight_data_list = []
    self.data_weight_mean = []
    self.data_weight_yerr = []

    self.data_length_mean = []
    self.data_length_yerr = []

    path = f"/---/---/---/---/---/{name}"
    self.simulation = pd.read_csv(path + f'/{name}_nanae.csv', header = None)

    for i in range(len(date)):
      self.filename = pd.read_csv(path + f'/{date[i]}_{name}.csv')
      date[i] = str(date[i])
      datedata = f'{date[i][4:6]}/{date[i][6:8]}'
      self.dates.append(datedata)


      if i == 0: #全数調査のためファイルの形が違うため
        pre_dte = datetime.datetime.strptime(date[i], '%Y%m%d')

        exp_data = self.filename
        exp_data = exp_data[exp_data['体重(g)'] != 'na'] # ここらへんは臨機応変に手で修正すること
        
        # 体重
        weightdata = exp_data['体重(g)']
        weightdata = weightdata.astype(float)
        self.weight_data_list.append(weightdata)
        self.data_weight_mean.append(weightdata.mean())

        self.data_weight_yerr.append(np.array(np.std(weightdata, ddof=1)))

        # 尾叉長
        lengthdata = exp_data['尾叉長(mm)']
        lengthdata = lengthdata.astype(float)

        self.data_length_mean.append(lengthdata.mean())

        self.data_length_yerr.append(np.array(np.std(lengthdata, ddof=1)))

      else:
        # 体重
        weightdata = self.filename
        weightdata = weightdata['weight[kg]']*1000
        self.weight_data_list.append(weightdata) #追加 
        self.data_weight_mean.append(weightdata.mean())
        self.data_weight_yerr.append(np.array(np.std(weightdata, ddof=1)))

        # 尾叉長
        lengthdata = self.filename
        lengthdata = lengthdata['fork length[m]']*1000 # m → mm
        self.data_length_mean.append(lengthdata.mean())
        self.data_length_yerr.append(np.array(np.std(lengthdata, ddof=1)))



      post_dte = datetime.datetime.strptime(date[i], '%Y%m%d')
      post_dte1 = date[i]
      period = post_dte - pre_dte
      self.periods.append(period.days)

  def weight_data(self):
    self.simulation = self.simulation[0:self.periods[len(self.periods)-1] + 10]
    self.simulation = self.simulation.T
    self.simulation_weight = self.simulation.iloc[0::4].dropna().reset_index(drop=True)
    self.sim_x = np.arange(self.periods[len(self.periods) -1] + 10)
    self.sim_y = np.array(self.simulation_weight.mean())
    self.simulation_weight_yerr = np.array(np.std(self.simulation_weight, ddof=1))
    simulation_weight_yerr_positive = [x + y for (x, y) in zip(self.sim_y, self.simulation_weight_yerr)] #combined1 = [x * y for (x, y) in zip(li1, li2)]
    self.simulation_weight_yerr_positive = np.array(simulation_weight_yerr_positive)
    simulation_weight_yerr_negative = [x - y for (x, y) in zip(self.sim_y, self.simulation_weight_yerr)] #combined1 = [x * y for (x, y) in zip(li1, li2)]
    self.simulation_weight_yerr_negative = np.array(simulation_weight_yerr_negative)

  def length_data(self):
    self.simulation = self.simulation[0:self.periods[len(self.periods)-1] + 10]
    self.simulation = self.simulation.T
    self.simulation_length = self.simulation.iloc[3::4].dropna().reset_index(drop=True)*10 #cm →mm
    self.sim_x = np.arange(self.periods[len(self.periods) -1] + 10)
    self.sim_y = np.array(self.simulation_length.mean())
    self.simulation_length_yerr = np.array(np.std(self.simulation_length, ddof=1))
    simulation_length_yerr_positive = [x + y for (x, y) in zip(self.sim_y, self.simulation_length_yerr)] #combined1 = [x * y for (x, y) in zip(li1, li2)]
    self.simulation_length_yerr_positive = np.array(simulation_length_yerr_positive)
    simulation_length_yerr_negative = [x - y for (x, y) in zip(self.sim_y, self.simulation_length_yerr)] #combined1 = [x * y for (x, y) in zip(li1, li2)]
    self.simulation_length_yerr_negative = np.array(simulation_length_yerr_negative)

  def feed_data(self):
    self.simulation = self.simulation[0:self.periods[len(self.periods)-1] + 10]
    self.simulation = self.simulation.T
    self.simulation_feed = self.simulation.iloc[1::4].dropna().reset_index(drop=True)*0.03351 #1粒あたりの餌の重さ Unity上のV_feed
    self.sim_x = np.arange(self.periods[len(self.periods) -1] + 10)
    self.sim_y = np.array(self.simulation_feed.mean())

  def energy_data(self):
    self.simulation = self.simulation[0:self.periods[len(self.periods)-1] + 10]
    self.simulation = self.simulation.T
    self.simulation_energy = self.simulation.iloc[1::3].dropna().reset_index(drop=True)
    self.sim_x = np.arange(self.periods[len(self.periods) -1] + 10)
    self.sim_y = np.array(self.simulation_energy.mean())
  
  def feed_efficacy_data(self, name):
    path = f"/---/---/---/---/---/{name}"
    self.ration = pd.read_csv(path + f'/{name}_onlyfeed_nanae_personal.csv', header = None)
    self.ration = self.ration.T

    self.data_feed_weight_mean = []
    self.data_feed_weight_yerr = []

    for i in range(len(self.periods)):
      ration_sum = 0

      max = self.ration.shape[1]

      for j in range(self.periods[i] + 1):
        if j <max:
          ration_sum += self.ration[j].values

        else:
          ration_sum += self.ration[max-1].values
      


      feedweight = ration_sum / self.weight_data_list[i] #ration_sumはもともと平均給餌量になっている
      print(feedweight, 'feed/weight')
      self.data_feed_weight_mean.append(feedweight.mean())
      print(feedweight.mean(), 'feed/weight.mean()')
      
      print(ration_sum, 'ration_sum')
      print(self.weight_data_list[i], f'weightdatamean[{i}]')
      self.data_feed_weight_yerr.append(np.array(np.std(feedweight, ddof=1)))
      print(np.array(np.std(feedweight, ddof=1)), 'yerr')


    self.simulation = self.simulation[0:self.periods[len(self.periods)-1] + 10]
    self.simulation = self.simulation.T
    self.simulation_feed = self.simulation.iloc[1::4].dropna().reset_index(drop=True)*0.03351 #1粒あたりの餌の重さ Unity上のV_feed

    self.sim_x = np.arange(self.periods[len(self.periods) -1] + 10)
    self.sim_y_feed = np.array(self.simulation_feed.mean())

    self.simulation_weight = self.simulation.iloc[0::4].dropna().reset_index(drop=True)
    self.sim_y_weight = np.array(self.simulation_weight.mean())

    self.sim_y_feed_weight = self.sim_y_feed / self.sim_y_weight
    self.sim_y_feed_weight = (self.simulation_feed / self.simulation_weight).mean()

    self.simulation_feed_weight = self.simulation_feed.truediv(self.simulation_weight)


  def weight_plot(self, flag=False):
    x = self.periods
    y = self.data_weight_mean

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('Weight')

    if flag:
      for i in range(len(self.simulation_weight)):
        ax.plot(self.sim_x, self.simulation_weight.iloc[i], color = 'lightslategray', alpha = 0.1, linewidth=1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    ax.plot(self.sim_x, self.sim_y, color = 'r', linewidth=2, label='sim_y_mean')
    ax.plot(self.sim_x, self.simulation_weight_yerr_positive, color = 'black', linewidth=1, label='sim_y_std')
    ax.plot(self.sim_x, self.simulation_weight_yerr_negative, color = 'black', linewidth=1)
    ax.scatter(x, y, label='exp_y_mean')
    # ax.plot(x, x * u[0] + u[1], 'r', label='least square')
    ax.set_ylim(0, 60)
    plt.xticks(x, self.dates)
    plt.xticks(rotation=45)
    ax.errorbar(x, self.data_weight_mean, yerr=self.data_weight_yerr, capsize=3, fmt='o', ecolor='k', ms=7, mfc='None', label='exp_errorbar')
    ax.legend(loc='best')

  def length_plot(self, flag=False):
    x = self.periods
    y = self.data_length_mean

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('length')

    if flag:
      for i in range(len(self.simulation_length)):
        ax.plot(self.sim_x, self.simulation_length.iloc[i], color = 'lightslategray', alpha = 0.1, linewidth=1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    ax.plot(self.sim_x, self.sim_y, color = 'r', linewidth=2, label='sim_y_mean')
    ax.plot(self.sim_x, self.simulation_length_yerr_positive, color = 'black', linewidth=1, label='sim_y_std')
    ax.plot(self.sim_x, self.simulation_length_yerr_negative, color = 'black', linewidth=1)
    ax.scatter(x, y, label='exp_y_mean')
    ax.set_ylim(0, 200)
    plt.xticks(x, self.dates)
    plt.xticks(rotation=45)
    ax.errorbar(x, self.data_length_mean, yerr=self.data_length_yerr, capsize=3, fmt='o', ecolor='k', ms=7, mfc='None', label='exp_errorbar')
    ax.legend(loc='best')


  def weight_plot_maxmin(self):
    x = self.periods
    y = self.data_weight_mean

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('Weight')

    weightmax_id = self.simulation_weight[0].idxmax()
    weightmin_id = self.simulation_weight[0].idxmin()

    for i in range(len(self.simulation_weight)):
      ax.plot(self.sim_x, self.simulation_weight.iloc[i], color = 'lightslategray', alpha = 0.1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html

    ax.plot(self.sim_x, self.simulation_weight.iloc[weightmax_id], color = 'r', label='sim_y_weightmax')
    ax.plot(self.sim_x, self.simulation_weight.iloc[weightmin_id], color = 'b',  label='sim_y_weightmin')
    ax.set_ylim(0, 60)
    plt.xticks(x, self.dates)
    plt.xticks(rotation=45)
    ax.legend(loc='best')

  def length_plot_maxmin(self):
    x = self.periods
    y = self.data_length_mean

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('length')
    lengthmax_id = self.simulation_length[0].idxmax()
    lengthmin_id = self.simulation_length[0].idxmin()

    for i in range(len(self.simulation_length)):
      ax.plot(self.sim_x, self.simulation_length.iloc[i], color = 'lightslategray', alpha = 0.1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    ax.plot(self.sim_x, self.simulation_length.iloc[lengthmax_id], color = 'r', label='sim_y_lengthmax')
    ax.plot(self.sim_x, self.simulation_length.iloc[lengthmin_id], color = 'b',  label='sim_y_lengthmin')
    ax.set_ylim(0, 200)
    plt.xticks(x, self.dates)
    plt.xticks(rotation=45)
    ax.legend(loc='best')


  def weight_table(self):
    df2 =pd.DataFrame(index = ['exp_体重の平均値(g)', 'exp_体重の標準偏差(g)'])
    for i in range(len(self.dates)):
      df2.loc['exp_体重の平均値(g)', self.dates[i]] = self.data_weight_mean[i] #locは行名 行番号ではない。　loc['行名', '列名']
      df2.loc['exp_体重の標準偏差(g)', self.dates[i]] = self.data_weight_yerr[i]
    display(df2)
    
    df1 =pd.DataFrame(index = ['sim_体重の平均値(g)', 'sim_体重の標準偏差(g)'])

    for i in self.periods:
      df1.loc['sim_体重の平均値(g)',f'{i}日目'] = self.sim_y[i]
      df1.loc['sim_体重の標準偏差(g)',f'{i}日目'] = self.simulation_weight_yerr[i]
    display(df1)

  def length_table(self):
    df2 =pd.DataFrame(index = ['exp_尾叉長の平均値(mm)', 'exp_尾叉長の標準偏差(mm)'])
    for i in range(len(self.dates)):
      df2.loc['exp_尾叉長の平均値(mm)', self.dates[i]] = self.data_length_mean[i]
      df2.loc['exp_尾叉長の標準偏差(mm)', self.dates[i]] = self.data_length_yerr[i]
    display(df2)

    df1 =pd.DataFrame(index = ['sim_尾叉長の平均値(mm)', 'sim_尾叉長の標準偏差(mm)'])

    for i in self.periods:
      df1.loc['sim_尾叉長の平均値(mm)',f'{i}日目'] = self.sim_y[i]
      df1.loc['sim_尾叉長の標準偏差(mm)',f'{i}日目'] = self.simulation_length_yerr[i]
    display(df1)

  def feed_efficacy(self):
    df2 =pd.DataFrame(index = [['exp_平均飼料効率(%)']])

    for i in range(len(self.periods)):
      ration_sum = 0
      max = self.ration.shape[1]

      if i == 0:
        df2.loc['exp_平均飼料効率(%)', self.dates[i]] = 'NaN'
      
      else:
        for j in range(self.periods[i]): # range(0)の時はfor文は回されていないので3/30のデータフレームがない。 こちらは+1しなくていい
          if j <max:
            ration_sum += self.ration[j].values

          else:
            ration_sum += self.ration[max-1].values

          df2.loc['exp_平均飼料効率(%)', self.dates[i]] = (self.data_weight_mean[i] - self.data_weight_mean[0]) / ration_sum

    display(df2)

    df1 =pd.DataFrame(index = [['sim_平均飼料効率(%)']])
    for i in (self.periods):
      
      if i == 0:
        df1.loc['sim_平均飼料効率(%)',f'{i}日目'] = 'NaN'
      
      else:
        df1.loc['sim_平均飼料効率(%)',f'{i}日目'] = (self.sim_y_weight[i] - self.sim_y_weight[0]) / self.sim_y_feed[i]
      
    display(df1)

  def feed_weight(self):
    df2 =pd.DataFrame(index = [['exp_体重あたりの平均給餌量(%)', 'exp_体重あたりの平均給餌量の標準偏差(%)']])

    #table
    for i in range(len(self.periods)):
      ration_sum = 0
      # print(ration_sum, '1回目')
      max = self.ration.shape[1]

      for j in range(self.periods[i] + 1): # range(0)の時はfor文は回されていないので3/30のデータフレームがない。→+1にすることでfor文で回すようにした。また、これをすることで、初期値でのfeed / weightができるようになった。feed / weightなので、1個先の餌量を取得しないといけない
        if j <max:
          ration_sum += self.ration[j].values

        else:
          ration_sum += self.ration[max-1].values


        df2.loc['exp_体重あたりの平均給餌量(%)', self.dates[i]] = (ration_sum / self.weight_data_list[i]).mean()
        df2.loc['exp_体重あたりの平均給餌量の標準偏差(%)', self.dates[i]] = np.array(np.std((ration_sum / self.weight_data_list[i]), ddof=1))



    display(df2)

    df1 =pd.DataFrame(index = [['sim_体重あたりの摂餌量(%)', 'sim_体重あたりの摂餌量の標準偏差(%)']])
    for i in (self.periods): #self.periods = [0, 51, 79, 106]
        df1.loc['sim_体重あたりの摂餌量(%)',f'{i}日目'] = (self.simulation_feed[i+1] / self.simulation_weight[i]).mean()
        df1.loc['sim_体重あたりの摂餌量の標準偏差(%)', f'{i}日目'] = np.array(np.std((self.simulation_feed[i+1] / self.simulation_weight[i]), ddof=1))


    display(df1)

    #plot
    x = self.periods
    y = self.data_feed_weight_mean

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('feed / weight')

    for i in range(len(self.simulation_feed_weight)):
      ax.plot(self.sim_x, self.simulation_feed_weight.iloc[i], color = 'lightslategray', alpha = 0.1, linewidth=1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    ax.plot(self.sim_x, self.sim_y_feed_weight, color = 'r', label='sim_y_mean')

    # ax.scatter(x, y, label='exp_y_mean')
    ax.errorbar(x, y, yerr=self.data_feed_weight_yerr, capsize=3, fmt='o', ecolor='k', ms=7, label='exp_data') # mfc='None'
    ax.set_ylim(0, 2)
    plt.xticks(x, self.dates)
    plt.xticks(rotation=45)
    ax.legend(loc='best') # これがないとラベルがちゃんと表示されなかった


    '''
    '''
    #plot
    x = self.periods
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('feed / weight')
    feedweightmax_id = self.simulation_feed_weight[98].idxmax()
    feedweightmin_id = self.simulation_feed_weight[98].idxmin()

    for i in range(len(self.simulation_feed_weight)):
      ax.plot(self.sim_x, self.simulation_feed_weight.iloc[i], color = 'lightslategray', alpha = 0.1, linewidth=1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    ax.plot(self.sim_x, self.sim_y_feed_weight, color = 'k', label='sim_y_mean')
    ax.plot(self.sim_x, self.simulation_feed_weight.iloc[feedweightmax_id], color = 'r', label='sim_y_feedmax')
    ax.plot(self.sim_x, self.simulation_feed_weight.iloc[feedweightmin_id], color = 'b',  label='sim_y_feedmin')
    ax.set_ylim(0, 2)
    plt.xticks(x, self.dates)
    # print(self.dates)
    plt.xticks(rotation=45)
    ax.legend(loc='best') # これがないとラベルがちゃんと表示されなかった

  def feed_plot(self):
    x = self.periods
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('feed')
    feedmax_id = self.simulation_feed[98].idxmax()
    feedmin_id = self.simulation_feed[98].idxmin()

    for i in range(len(self.simulation_feed)):
      ax.plot(self.sim_x, self.simulation_feed.iloc[i], color = 'lightslategray', alpha = 0.1, linewidth=1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    ax.plot(self.sim_x, self.sim_y, color = 'k', label='sim_y_mean')
    ax.plot(self.sim_x, self.simulation_feed.iloc[feedmax_id], color = 'r', label='sim_y_feedmax')
    ax.plot(self.sim_x, self.simulation_feed.iloc[feedmin_id], color = 'b',  label='sim_y_feedmin')
    ax.set_ylim(0, 80)
    plt.xticks(x, self.dates)
    plt.xticks(rotation=45)
    ax.legend(loc='best') # これがないとラベルがちゃんと表示されなかった

  def energy_plot(self):
    x = self.periods
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('feed')

    energymax_id = self.simulation_energy[98].idxmax()
    energymin_id = self.simulation_energy[98].idxmin()

    for i in range(len(self.simulation_energy)):
      ax.plot(self.sim_x, self.simulation_energy.iloc[i], color = 'lightslategray', alpha = 0.1, linewidth=1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html

    ax.plot(self.sim_x, self.sim_y, color = 'k', label='sim_y_mean')
    ax.plot(self.sim_x, self.simulation_energy.iloc[energymax_id], color = 'r', label='sim_y_feedmax')
    ax.plot(self.sim_x, self.simulation_energy.iloc[energymin_id], color = 'b',  label='sim_y_feedmin')
    ax.set_ylim(0, 8000)
    plt.xticks(x, self.dates)
    plt.xticks(rotation=45)
    ax.legend(loc='best') # これがないとラベルがちゃんと表示されなかった

# サクラマス(cherry trout/cherry salmon)

In [ ]:
class Model_sakuramasu:

  def __init__(self, name, date):
    self.periods = []
    self.dates = []
    self.weight_data_list = []
    self.data_weight_mean = []
    self.data_weight_yerr = []

    self.data_length_mean = []
    self.data_length_yerr = []
    path = f"/---/---/---/---/---/{name}"
    self.simulation = pd.read_csv(path + f'/{name}_nanae.csv', header = None)
    # self.simulation = pd.read_csv(path + f'/{name}_nanae_fail.csv', header = None)
    # display(self.simulation)

    for i in range(len(date)):
      self.filename = pd.read_csv(path + f'/{date[i]}_{name}.csv')
      date[i] = str(date[i])
      datedata = f'{date[i][4:6]}/{date[i][6:8]}'
      self.dates.append(datedata)

      # data_weight_mean = self.filnename.mean()

      if i == 0:
        pre_dte = datetime.datetime.strptime(date[i], '%Y%m%d')

        exp_data = self.filename
        exp_data = exp_data[exp_data['尾叉長'] != 'Na'] # ここらへんは臨機応変に手で修正すること
        
        # 体重
        weightdata = exp_data['体重']
        weightdata = weightdata.astype(float)
        self.weight_data_list.append(weightdata) #体重データ
        # expデータの体重の平均値
        self.data_weight_mean.append(weightdata.mean())
        # expデータの体重の不偏標準偏差
        self.data_weight_yerr.append(np.array(np.std(weightdata, ddof=1)))

        # 尾叉長
        lengthdata = exp_data['尾叉長']
        lengthdata = lengthdata.astype(float)
        # expデータの体重の平均値
        self.data_length_mean.append(lengthdata.mean())
        # expデータの体重の不偏標準偏差
        self.data_length_yerr.append(np.array(np.std(lengthdata, ddof=1)))

      else:
        # 体重
        weightdata = self.filename
        weightdata = weightdata['weight[kg]']*1000 #kg → g
        self.weight_data_list.append(weightdata) #体重データ
        self.data_weight_mean.append(weightdata.mean())
        self.data_weight_yerr.append(np.array(np.std(weightdata, ddof=1)))

        # 尾叉長
        lengthdata = self.filename
        lengthdata = lengthdata['fork length[m]']*1000 # m → mm
        self.data_length_mean.append(lengthdata.mean())
        self.data_length_yerr.append(np.array(np.std(lengthdata, ddof=1)))

        # display(weightdata)

      # print(self.data_weight_mean)
      # print(i)

      post_dte = datetime.datetime.strptime(date[i], '%Y%m%d')
      post_dte1 = date[i]
      period = post_dte - pre_dte
      self.periods.append(period.days)
      # print(period.days)

      # display(filename)
    
    # display(simulation)
    # return periods, dates

  def weight_data(self):
    self.simulation = self.simulation[0:self.periods[len(self.periods)-1] + 10]
    self.simulation = self.simulation.T
    self.simulation_weight = self.simulation.iloc[0::4].dropna().reset_index(drop=True)
    self.sim_x = np.arange(self.periods[len(self.periods) -1] + 10)
    self.sim_y = np.array(self.simulation_weight.mean())
    self.simulation_weight_yerr = np.array(np.std(self.simulation_weight, ddof=1))
    simulation_weight_yerr_positive = [x + y for (x, y) in zip(self.sim_y, self.simulation_weight_yerr)] #combined1 = [x * y for (x, y) in zip(li1, li2)]
    self.simulation_weight_yerr_positive = np.array(simulation_weight_yerr_positive)
    simulation_weight_yerr_negative = [x - y for (x, y) in zip(self.sim_y, self.simulation_weight_yerr)] #combined1 = [x * y for (x, y) in zip(li1, li2)]
    self.simulation_weight_yerr_negative = np.array(simulation_weight_yerr_negative)

  
  def length_data(self):
    self.simulation = self.simulation[0:self.periods[len(self.periods)-1] + 10]
    self.simulation = self.simulation.T
    self.simulation_length = self.simulation.iloc[3::4].dropna().reset_index(drop=True)*10 #cm →mm
    self.sim_x = np.arange(self.periods[len(self.periods) -1] + 10)
    self.sim_y = np.array(self.simulation_length.mean())
    self.simulation_length_yerr = np.array(np.std(self.simulation_length, ddof=1))
    simulation_length_yerr_positive = [x + y for (x, y) in zip(self.sim_y, self.simulation_length_yerr)] #combined1 = [x * y for (x, y) in zip(li1, li2)]
    self.simulation_length_yerr_positive = np.array(simulation_length_yerr_positive)
    simulation_length_yerr_negative = [x - y for (x, y) in zip(self.sim_y, self.simulation_length_yerr)] #combined1 = [x * y for (x, y) in zip(li1, li2)]
    self.simulation_length_yerr_negative = np.array(simulation_length_yerr_negative)

  def feed_data(self):
    self.simulation = self.simulation[0:self.periods[len(self.periods)-1] + 10]
    self.simulation = self.simulation.T
    self.simulation_feed = self.simulation.iloc[1::4].dropna().reset_index(drop=True)*0.03351 #1粒あたりの餌の重さ Unity上のV_feed
    # self.simulation_feed = self.simulation.iloc[1::4].dropna().reset_index(drop=True)
    self.sim_x = np.arange(self.periods[len(self.periods) -1] + 10)
    self.sim_y = np.array(self.simulation_feed.mean())

  def energy_data(self):
    self.simulation = self.simulation[0:self.periods[len(self.periods)-1] + 10]
    self.simulation = self.simulation.T
    self.simulation_energy = self.simulation.iloc[1::3].dropna().reset_index(drop=True)
    self.sim_x = np.arange(self.periods[len(self.periods) -1] + 10)
    self.sim_y = np.array(self.simulation_energy.mean())


  def feed_efficacy_data(self, name):
    path = f"/content/drive/My Drive/takahashi_research/data/{name}"
    self.ration = pd.read_csv(path + f'/{name}_onlyfeed_nanae_personal.csv', header = None)
    self.ration = self.ration.T
    # display(self.ration)

    self.data_feed_weight_mean = []
    self.data_feed_weight_yerr = []

    for i in range(len(self.periods)):
      # print(i)
      # print(self.data_weight_mean[i], f'data_weight_mean[{i}]')
      ration_sum = 0

      max = self.ration.shape[1]
      # print(max)

      for j in range(self.periods[i] + 1):
        if j <max:
          ration_sum += self.ration[j].values

        else:
          ration_sum += self.ration[max-1].values
      
      # print(ration_sum, 'ration_sum')


      feedweight = ration_sum / self.weight_data_list[i] #ration_sumはもともと平均給餌量になっている
      print(feedweight, 'feed/weight')
      self.data_feed_weight_mean.append(feedweight.mean())
      print(feedweight.mean(), 'feed/weight.mean()')
      
      print(ration_sum, 'ration_sum')
      print(self.weight_data_list[i], f'weightdatamean[{i}]')
      # display(self.weight_data_list[i])
      # print(feedweight.mean(), 'feedweightmean')
      self.data_feed_weight_yerr.append(np.array(np.std(feedweight, ddof=1)))
      print(np.array(np.std(feedweight, ddof=1)), 'yerr')


    self.simulation = self.simulation[0:self.periods[len(self.periods)-1] + 10]
    self.simulation = self.simulation.T
    self.simulation_feed = self.simulation.iloc[1::4].dropna().reset_index(drop=True)*0.03351 #1粒あたりの餌の重さ Unity上のV_feed
    # self.simulation_feed = self.simulation_feed.drop(0, axis=1)

    # print(self.simulation_feed)
    # print(self.simulation_feed.shape)
    # self.simulation_feed = self.simulation.iloc[1::4].dropna().reset_index(drop=True)
    self.sim_x = np.arange(self.periods[len(self.periods) -1] + 10)
    self.sim_y_feed = np.array(self.simulation_feed.mean())

    self.simulation_weight = self.simulation.iloc[0::4].dropna().reset_index(drop=True)
    self.sim_y_weight = np.array(self.simulation_weight.mean())

    self.sim_y_feed_weight = self.sim_y_feed / self.sim_y_weight
    self.sim_y_feed_weight = (self.simulation_feed / self.simulation_weight).mean()

    self.simulation_feed_weight = self.simulation_feed.truediv(self.simulation_weight)

  def weight_plot(self, flag=False):
    x = self.periods
    # print(x)
    y = self.data_weight_mean
    # print(y)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('Weight')
    # print(self.sim_x)
    # print(self.sim_y)
    if flag:
      for i in range(len(self.simulation_weight)):
        ax.plot(self.sim_x, self.simulation_weight.iloc[i], color = 'lightslategray', alpha = 0.1, linewidth=1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    ax.plot(self.sim_x, self.sim_y, color = 'r', linewidth=2, label='sim_y_mean')
    ax.plot(self.sim_x, self.simulation_weight_yerr_positive, color = 'black', linewidth=1, label='sim_y_std')
    ax.plot(self.sim_x, self.simulation_weight_yerr_negative, color = 'black', linewidth=1)
    ax.scatter(x, y, label='exp_y_mean')
    # ax.plot(x, x * u[0] + u[1], 'r', label='least square')
    ax.set_ylim(0, 60)
    plt.xticks(x, self.dates)
    # print(self.dates)
    plt.xticks(rotation=45)
    ax.errorbar(x, self.data_weight_mean, yerr=self.data_weight_yerr, capsize=3, fmt='o', ecolor='k', ms=7, mfc='None', label='exp_errorbar')
    ax.legend(loc='best')

  def length_plot(self, flag=False):
    x = self.periods # ここにシミュレーションの日数が入っている。
    # print(x)
    y = self.data_length_mean
    # print(y)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('length')
    # print(self.sim_x)
    # print(self.sim_y)
    if flag:
      for i in range(len(self.simulation_length)):
        ax.plot(self.sim_x, self.simulation_length.iloc[i], color = 'lightslategray', alpha = 0.1, linewidth=1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    ax.plot(self.sim_x, self.sim_y, color = 'r', linewidth=2, label='sim_y_mean')
    ax.plot(self.sim_x, self.simulation_length_yerr_positive, color = 'black', linewidth=1, label='sim_y_std')
    ax.plot(self.sim_x, self.simulation_length_yerr_negative, color = 'black', linewidth=1)
    ax.scatter(x, y, label='exp_y_mean')
    # ax.plot(x, x * u[0] + u[1], 'r', label='least square')
    ax.set_ylim(0, 200)
    plt.xticks(x, self.dates)
    # print(self.dates)
    plt.xticks(rotation=45)
    ax.errorbar(x, self.data_length_mean, yerr=self.data_length_yerr, capsize=3, fmt='o', ecolor='k', ms=7, mfc='None', label='exp_errorbar')
    ax.legend(loc='best')

  
  def weight_plot_maxmin(self):
    x = self.periods
    # print(x)
    y = self.data_weight_mean
    # print(y)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('Weight')
    # display(self.simulation_weight)
    weightmax_id = self.simulation_weight[98].idxmax()
    weightmin_id = self.simulation_weight[98].idxmin()
    # print(len(self.sim_x))
    # print(len(self.sim_y[weightmax_id]))
    # print(self.simulation_weight.iloc[weightmax_id]) #行番号
    for i in range(len(self.simulation_weight)):
      ax.plot(self.sim_x, self.simulation_weight.iloc[i], color = 'lightslategray', alpha = 0.1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    # ax.scatter(self.sim_x, self.simulation_weight.iloc[weightmax_id], color = 'r', s=10, label='sim_y_weightmax')
    ax.plot(self.sim_x, self.simulation_weight.iloc[weightmax_id], color = 'r', label='sim_y[lastdays]_weightmax')
    # ax.scatter(self.sim_x, self.simulation_weight.iloc[weightmin_id], color = 'b', s=10, label='sim_y_weightmin')
    ax.plot(self.sim_x, self.simulation_weight.iloc[weightmin_id], color = 'b',  label='sim_y[lastdays]_weightmin')
    ax.set_ylim(0, 80)
    plt.xticks(x, self.dates)
    plt.xticks(rotation=45)
    ax.legend(loc='best')

  def length_plot_maxmin(self):
    x = self.periods
    # print(x)
    y = self.data_length_mean
    # print(y)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('length')
    # display(self.simulation_length)
    lengthmax_id = self.simulation_length[0].idxmax()
    lengthmin_id = self.simulation_length[0].idxmin()
    # print(len(self.sim_x))
    # print(len(self.sim_y[lengthmax_id]))
    # print(self.simulation_length.iloc[lengthmax_id]) #行番号
    for i in range(len(self.simulation_length)):
      ax.plot(self.sim_x, self.simulation_length.iloc[i], color = 'lightslategray', alpha = 0.1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    # ax.scatter(self.sim_x, self.simulation_length.iloc[lengthmax_id], color = 'r', s=10, label='sim_y_lengthmax')
    ax.plot(self.sim_x, self.simulation_length.iloc[lengthmax_id], color = 'r', label='sim_y_lengthmax')
    # ax.scatter(self.sim_x, self.simulation_length.iloc[lengthmin_id], color = 'b', s=10, label='sim_y_lengthmin')
    ax.plot(self.sim_x, self.simulation_length.iloc[lengthmin_id], color = 'b',  label='sim_y_lengthmin')
    ax.set_ylim(0, 200)
    plt.xticks(x, self.dates)
    plt.xticks(rotation=45)
    ax.legend(loc='best')

  def weight_table(self):
    df2 =pd.DataFrame(index = ['exp_体重の平均値(g)', 'exp_体重の標準偏差(g)'])
    for i in range(len(self.dates)):
      # print(self.dates[i])
      # print(self.data_weight_mean[i])
      df2.loc['exp_体重の平均値(g)', self.dates[i]] = self.data_weight_mean[i] #locは行名
      df2.loc['exp_体重の標準偏差(g)', self.dates[i]] = self.data_weight_yerr[i]
    display(df2)
    
    df1 =pd.DataFrame(index = ['sim_体重の平均値(g)', 'sim_体重の標準偏差(g)'])
    # display(df1)
    for i in self.periods:
      # display(self.sim_y[i])
      # display(self.simulation_weight_yerr[i])
      # df['平均値'] = self.sim_y[i]
      df1.loc['sim_体重の平均値(g)',f'{i}日目'] = self.sim_y[i]
      df1.loc['sim_体重の標準偏差(g)',f'{i}日目'] = self.simulation_weight_yerr[i]
    display(df1)

  def length_table(self):
    df2 =pd.DataFrame(index = ['exp_尾叉長の平均値(mm)', 'exp_尾叉長の標準偏差(mm)'])
    for i in range(len(self.dates)):
      # print(self.dates[i])
      # print(self.data_length_mean[i])
      df2.loc['exp_尾叉長の平均値(mm)', self.dates[i]] = self.data_length_mean[i]
      df2.loc['exp_尾叉長の標準偏差(mm)', self.dates[i]] = self.data_length_yerr[i]
    display(df2)

    df1 =pd.DataFrame(index = ['sim_尾叉長の平均値(mm)', 'sim_尾叉長の標準偏差(mm)'])
    # display(df1)
    for i in self.periods:
      # display(self.sim_y[i])
      # display(self.simulation_length_yerr[i])
      # df['平均値'] = self.sim_y[i]
      df1.loc['sim_尾叉長の平均値(mm)',f'{i}日目'] = self.sim_y[i]
      df1.loc['sim_尾叉長の標準偏差(mm)',f'{i}日目'] = self.simulation_length_yerr[i]
    display(df1)

  def feed_efficacy(self):
    df2 =pd.DataFrame(index = [['exp_飼料効率の平均値(%)']])
    # for i in range(len(self.dates)):
    for i in range(len(self.periods)):
      ration_sum = 0
      # print(ration_sum, '1回目')
      max = self.ration.shape[1]

      for j in range(self.periods[i]):
        if j <max:
          ration_sum += self.ration[j].values

        else:
          ration_sum += self.ration[max-1].values

        # print(ration_sum, '2回目')

        df2.loc['exp_飼料効率の平均値(%)', self.dates[i]] =  (self.data_weight_mean[i] - self.data_weight_mean[0]) / ration_sum



    display(df2)

    df1 =pd.DataFrame(index = [['sim_飼料効率の平均値(%)']])
    for i in (self.periods):

      if i >0:
        df1.loc['sim_飼料効率の平均値(%)',f'{i}日目'] = (self.sim_y_weight[i] - self.sim_y_weight[0])/self.sim_y_feed[i]
        
    display(df1)


  def feed_plot(self):
    x = self.periods
    # print(x)
    # y = self.data_feed_mean
    # print(y)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('feed')
    # print(self.simulation_feed[0])
    # print(self.simulation_feed)
    feedmax_id = self.simulation_feed[98].idxmax()
    feedmin_id = self.simulation_feed[98].idxmin()
    # print( feedmax_id)
    # print( feedmin_id)
    # print(self.sim_x)
    # print(self.sim_y)
    # if flag:
    for i in range(len(self.simulation_feed)):
      ax.plot(self.sim_x, self.simulation_feed.iloc[i], color = 'lightslategray', alpha = 0.1, linewidth=1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    ax.plot(self.sim_x, self.sim_y, color = 'k', label='sim_y_mean')
    ax.plot(self.sim_x, self.simulation_feed.iloc[feedmax_id], color = 'r', label='sim_y_feedmax')
    ax.plot(self.sim_x, self.simulation_feed.iloc[feedmin_id], color = 'b',  label='sim_y_feedmin')
    # ax.set_ylim(0, 200)
    # ax.set_ylim(0, 1200)
    ax.set_ylim(0, 80)
    # ax.set_ylim(0, 10)
    plt.xticks(x, self.dates)
    # print(self.dates)
    plt.xticks(rotation=45)

  def energy_plot(self):
    x = self.periods
    # print(x)
    # y = self.data_feed_mean
    # print(y)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('feed')
    # print(self.simulation_feed[0])
    # print(self.simulation_feed)
    energymax_id = self.simulation_energy[98].idxmax()
    energymin_id = self.simulation_energy[98].idxmin()
    # print( feedmax_id)
    # print( feedmin_id)
    # print(self.sim_x)
    # print(self.sim_y)
    # if flag:
    for i in range(len(self.simulation_energy)):
      ax.plot(self.sim_x, self.simulation_energy.iloc[i], color = 'lightslategray', alpha = 0.1, linewidth=1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    ax.plot(self.sim_x, self.sim_y, color = 'k', label='sim_y_mean')
    ax.plot(self.sim_x, self.simulation_energy.iloc[energymax_id], color = 'r', label='sim_y_feedmax')
    ax.plot(self.sim_x, self.simulation_energy.iloc[energymin_id], color = 'b',  label='sim_y_feedmin')
    # ax.set_ylim(0, 200)
    # ax.set_ylim(0, 1200)
    ax.set_ylim(0, 80)
    plt.xticks(x, self.dates)
    # print(self.dates)
    plt.xticks(rotation=45)
    ax.legend(loc='best') # これがないとラベルがちゃんと表示されなかった

  def feed_efficacy(self):
    df2 =pd.DataFrame(index = [['exp_平均飼料効率(%)']])
    # for i in range(len(self.dates)):
    for i in range(len(self.periods)):
      ration_sum = 0
      # print(ration_sum, '1回目')
      max = self.ration.shape[1]

      if i == 0:
        df2.loc['exp_平均飼料効率(%)', self.dates[i]] = 'NaN'
      
      else:
        for j in range(self.periods[i]): # range(0)の時はfor文は回されていないので3/30のデータフレームがない。 こちらは+1しなくていい
          if j <max:
            ration_sum += self.ration[j].values

          else:
            ration_sum += self.ration[max-1].values

        # try:
        #   for j in range(self.periods[i]):
        #     # print(self.ration[i])
        #     ration_sum = ration_sum + self.ration[j].values
        
        # except:
        #   print(ration_sum)
        #   for j in range(self.periods[i]):
        #     ration_sum = ration_sum + self.ration[j].values
          
        #   print(ration_sum)

            # print(ration_sum)


          # print(ration_sum, '2回目')

          df2.loc['exp_平均飼料効率(%)', self.dates[i]] = (self.data_weight_mean[i] - self.data_weight_mean[0]) / ration_sum

    display(df2)

    df1 =pd.DataFrame(index = [['sim_平均飼料効率(%)']])
    for i in (self.periods):
      # df1.loc['sim_飼料効率の平均値(%)',f'{i}日目'] = (self.sim_y_weight[i] - self.sim_y_weight[0]) / self.sim_y_feed[i]
      
      if i == 0:
        df1.loc['sim_平均飼料効率(%)',f'{i}日目'] = 'NaN'
      
      else:
        df1.loc['sim_平均飼料効率(%)',f'{i}日目'] = (self.sim_y_weight[i] - self.sim_y_weight[0]) / self.sim_y_feed[i]
      
    display(df1)

  def feed_weight(self):
    df2 =pd.DataFrame(index = [['exp_体重あたりの平均給餌量(%)', 'exp_体重あたりの平均給餌量の標準偏差(%)']])
    # for i in range(len(self.dates)):

    #table
    for i in range(len(self.periods)):
      ration_sum = 0
      # print(ration_sum, '1回目')
      max = self.ration.shape[1]

      # if i == 0:
      #   df2.loc['exp_平均給餌量', self.dates[i]] = ration_sum / self.data_weight_mean[i]

      for j in range(self.periods[i] + 1): # range(0)の時はfor文は回されていないので3/30のデータフレームがない。→+1にすることでfor文で回すようにした。また、これをすることで、初期値でのfeed / weightができるようになった。feed / weightなので、1個先の餌量を取得しないといけない
        if j <max:
          ration_sum += self.ration[j].values

        else:
          ration_sum += self.ration[max-1].values

      # try:
      #   for j in range(self.periods[i]):
      #     # print(self.ration[i])
      #     ration_sum = ration_sum + self.ration[j].values
      
      # except:
      #   print(ration_sum)
      #   for j in range(self.periods[i]):
      #     ration_sum = ration_sum + self.ration[j].values
        
      #   print(ration_sum)

          # print(ration_sum)


        # print(ration_sum, '2回目')

        df2.loc['exp_体重あたりの平均給餌量(%)', self.dates[i]] = (ration_sum / self.weight_data_list[i]).mean()
        df2.loc['exp_体重あたりの平均給餌量の標準偏差(%)', self.dates[i]] = np.array(np.std((ration_sum / self.weight_data_list[i]), ddof=1))



    display(df2)

    df1 =pd.DataFrame(index = [['sim_体重あたりの摂餌量(%)', 'sim_体重あたりの摂餌量の標準偏差(%)']])
    for i in (self.periods): #self.periods = [0, 51, 79, 106]
        df1.loc['sim_体重あたりの摂餌量(%)',f'{i}日目'] = (self.simulation_feed[i+1] / self.simulation_weight[i]).mean()
        df1.loc['sim_体重あたりの摂餌量の標準偏差(%)', f'{i}日目'] = np.array(np.std((self.simulation_feed[i+1] / self.simulation_weight[i]), ddof=1))


    display(df1)

    #plot
    x = self.periods
    # print(x)
    y = self.data_feed_weight_mean
    # print(y)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('feed / weight')
    # print(self.simulation_feed[0])
    # print(self.simulation_feed)
    # feedweightmax_id = self.simulation_feed_weight[98].idxmax()
    # feedweightmin_id = self.simulation_feed_weight[98].idxmin()
    # # print( feedmax_id)
    # print( feedmin_id)
    # print(self.sim_x)
    # print(self.sim_y)
    # if flag:

    for i in range(len(self.simulation_feed_weight)):
      ax.plot(self.sim_x, self.simulation_feed_weight.iloc[i], color = 'lightslategray', alpha = 0.1, linewidth=1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    ax.plot(self.sim_x, self.sim_y_feed_weight, color = 'r', label='sim_y_mean')

    # ax.scatter(x, y, label='exp_y_mean')
    ax.errorbar(x, y, yerr=self.data_feed_weight_yerr, capsize=3, fmt='o', ecolor='k', ms=7, label='exp_data') #mfc = 'None' ●が白抜きになる

    # ax.plot(self.sim_x, self.simulation_feed_weight.iloc[feedweightmax_id], color = 'r', label='sim_y_feedmax')
    # ax.plot(self.sim_x, self.simulation_feed_weight.iloc[feedweightmin_id], color = 'b',  label='sim_y_feedmin')
    # ax.set_ylim(0, 200)
    # ax.set_ylim(0, 1200)
    ax.set_ylim(0, 2)
    plt.xticks(x, self.dates)
    # print(self.dates)
    plt.xticks(rotation=45)
    ax.legend(loc='best') # これがないとラベルがちゃんと表示されなかった


    '''
    '''
    #plot
    x = self.periods
    # print(x)
    # y = self.data_feed_mean
    # print(y)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('date')
    ax.set_ylabel('feed / weight')
    # print(self.simulation_feed[0])
    # print(self.simulation_feed)
    feedweightmax_id = self.simulation_feed_weight[98].idxmax()
    feedweightmin_id = self.simulation_feed_weight[98].idxmin()
    # # print( feedmax_id)
    # print( feedmin_id)
    # print(self.sim_x)
    # print(self.sim_y)
    # if flag:
    for i in range(len(self.simulation_feed_weight)):
      ax.plot(self.sim_x, self.simulation_feed_weight.iloc[i], color = 'lightslategray', alpha = 0.1, linewidth=1) #alpha 透明度, 色の参照URL https://matplotlib.org/stable/gallery/color/named_colors.html
    ax.plot(self.sim_x, self.sim_y_feed_weight, color = 'k', label='sim_y_mean')
    ax.plot(self.sim_x, self.simulation_feed_weight.iloc[feedweightmax_id], color = 'r', label='sim_y_feedmax')
    ax.plot(self.sim_x, self.simulation_feed_weight.iloc[feedweightmin_id], color = 'b',  label='sim_y_feedmin')
    # ax.set_ylim(0, 200)
    # ax.set_ylim(0, 1200)
    ax.set_ylim(0, 2)
    plt.xticks(x, self.dates)
    # print(self.dates)
    plt.xticks(rotation=45)
    ax.legend(loc='best') # これがないとラベルがちゃんと表示されなかった




# 構築

In [ ]:
name = 'rainbowtrout'

In [ ]:
date = [20220330, 20220520, 20220617, 20220714]

In [ ]:
# filename, dates = Model_rainbowtrout(name, date)
# filename, dates
# for i in filesname:
#   i.describe()

In [ ]:
data = Model_rainbowtrout(name, date)
data.weight_data()
data.weight_plot() # True で全個体をplotする

In [ ]:
data = Model_rainbowtrout(name, date)
data.weight_data()
data.weight_plot_maxmin()

In [ ]:
data = Model_rainbowtrout(name, date)
data.weight_data()
data.weight_table()

In [ ]:
data = Model_rainbowtrout(name, date)
data.length_data()
data.length_plot()

In [ ]:
data = Model_rainbowtrout(name, date)
data.length_data()
data.length_plot_maxmin()

In [ ]:
data = Model_rainbowtrout(name, date)
data.length_data()
data.length_table()

In [ ]:
data = Model_rainbowtrout(name, date)
data.feed_data()
data.feed_plot()

In [ ]:
data = Model_rainbowtrout(name, date)
data.energy_data()
data.energy_plot()

In [ ]:
data = Model_rainbowtrout(name, date)
data.feed_efficacy_data(name)
data.feed_efficacy()
data.feed_weight()

In [ ]:
name = 'sakuramasu'
date = [20220330, 20220502, 20220520, 20220606, 20220617, 20220627]

In [ ]:
data = Model_sakuramasu(name, date)
data.weight_data()
data.weight_plot()

In [ ]:
data = Model_sakuramasu(name, date)
data.weight_data()
data.weight_plot_maxmin()

In [ ]:
data = Model_sakuramasu(name, date)
data.weight_data()
data.weight_table()

In [ ]:
data = Model_sakuramasu(name, date)
data.length_data()
data.length_plot()

In [ ]:
data = Model_sakuramasu(name, date)
data.length_data()
data.length_plot_maxmin()

In [ ]:
data = Model_sakuramasu(name, date)
data.length_data()
data.length_table()

In [ ]:
data = Model_sakuramasu(name, date)
data.feed_data()
data.feed_plot()

In [ ]:
data = Model_sakuramasu(name, date)
data.energy_data()
data.energy_plot()

In [ ]:
data = Model_sakuramasu(name, date)
data.feed_efficacy_data(name)
data.feed_efficacy()
data.feed_weight()